In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast
from fuzzywuzzy import process
from nltk.corpus import words

## Initial Experiments

In [ ]:
total = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\total_summary_stats_updated.csv")

In [ ]:
patient_id = total["patient_id"].to_list()
patient_id = patient_id[:10]

In [ ]:
matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches.csv")

In [ ]:
# ONLY RUN ONCE #

# vertifying that all match records can be found
for i in range(len(patient_id)):
    specific_matches = matches[matches["PatientID"] == patient_id[i]]
    if (len(specific_matches) <= 0):
        print("NOT FOUND: ", patient_id[i])
    #specific_matches.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_{}.csv".format(i+1))

In [ ]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\Specific_Matches\patient_1.csv")

In [ ]:
unique_ids = matches["PatientID"].unique().tolist()
print(len(unique_ids))

In [ ]:
# interesting match cases
# patient_2 in specific study row 6, cream that has mci inside it

In [ ]:
print(subset_loc_clone)

## Final Code

In [ ]:
p1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_matches.csv")

In [ ]:
p1.head()

In [ ]:
total_merged = []

for index, row in p1.iterrows():
    curr = str(row["regex_location"])
    curr = curr[1:len(curr) - 1]
    curr = list((eval(curr)))
    #print("CURR: ", len(curr))
    # print(curr)
    merged = []

    start = 0
    end = 0
    length = 0
    
    if (len(curr) == 2 and isinstance(curr[1], int)):
        start = curr[0]
        end = curr[1]
        #print(start, end)
        length = len(row["NoteTXT"])
        if (start < 100):
            start = 100
        elif (end > length - 100):
            end = length - 100
        curr = [(start - 100, end + 100)]
        
    else:
        loc_list = []
        for j in range(len(curr)):
            start = curr[j][0]
            end = curr[j][1]
            loc_list.append((start, end))
        length = len(row["NoteTXT"])
        for k in range(len(loc_list)):
            start = loc_list[k][0]
            end = loc_list[k][1]
            if (start < 100):
                start = 100
            elif (end > length - 100):
                end = length - 100
            loc_list[k] = (start - 100, end + 100)
        curr = loc_list
        
    for interval in curr:
        if not merged:
            merged.append(interval)
            continue

        prevInterval = merged.pop()
        if prevInterval[0] <= interval[0] <= prevInterval[1]:
            startTime = prevInterval[0]
            endTime = prevInterval[1] if prevInterval[1] > interval[1] else interval[1]
            merged.append((startTime, endTime))
        else:
            merged.append(prevInterval)
            merged.append(interval)
            
    total_merged.append(merged)
p1["merged_row_location"] = total_merged

In [ ]:
p1

In [ ]:
total_merged_note_txt = []
for index, row in p1.iterrows():
    curr = row["merged_row_location"]
    merged_note_txt = ""
    
    for i in range(len(curr)):
        start = curr[i][0]
        end = curr[i][1]
        
        merged_note_txt += " ------- " + row["NoteTXT"][start:end]
    total_merged_note_txt.append(merged_note_txt)
    
p1["merged_regex_sent"] = total_merged_note_txt

In [ ]:
len(p1)

In [ ]:
p1['merged_regex_sent'] = p1['merged_regex_sent'].str.split(' ',1).str[1].str.rpartition(' ')[0]

In [ ]:
#p1['NoteTXT'] = p1['NoteTXT'].astype(str)
p1 = p1.drop_duplicates(subset='merged_regex_sent', keep="first")
p1 = p1.replace(r'^\s*$', np.NaN, regex=True)
p1.reset_index(drop = True)

In [ ]:
p1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_matches_updated.csv", index = False)

## Getting #  of Unique Encounters per Patient

In [58]:
## getting number of unique encounters per patient
df = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_matches_updated.csv")

In [59]:
df = df.drop_duplicates(subset = 'PatientEncounterID', keep = "first")

In [60]:
summary_stats_1 = df.groupby(['EMPI'])["PatientEncounterID"].count().reset_index()
summary_stats_1

,EMPI,PatientEncounterID
0,106692571,1
1,106693931,5
2,106694114,15
3,106695043,6
4,106695334,1
...,...,...
4100,113714475,9
4101,113716850,6
4102,113722466,5
4103,113722742,17


In [61]:
summary_stats_1.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\5\apoe_5_summary_stats_updated.csv")

## Cleaning up Similar Encounters on Patient Level

In [62]:
df = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\1\apoe_1_matches_updated.csv")

In [63]:
df

,Unnamed: 0,Unnamed: 0.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,buckets,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,merged_row_location,merged_regex_sent
0,0,0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,471433064,Progress Notes,1,4.532520e+08,1.0,----------------------------------------------...,"[(5850, 5858)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1,"[(5750, 5958)]",------- ncer. She passed secondary to this. He...
1,1,1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,513142600,Progress Notes,1,4.924833e+08,1.0,----------------------------------------------...,"[(3643, 3649)]",'a. Skin: No rashes or lesions. Neurological: ...,['memory'],1,"[(3543, 3749)]",------- a. Skin: No rashes or lesions. Neurolo...
2,13,13,100000272,10040002098,Z6352398,3.161459e+09,2017-06-23 00:00:00.0000000,Hospital Encounter,1562613299,ED Provider Notes,1,1.508556e+09,1.0,----------------------------------------------...,"[(3947, 3950)]",'cular: Normal rate. Pulmonary/Chest: Effort n...,['mci'],1,"[(3847, 4050)]",------- cular: Normal rate. Pulmonary/Chest: E...
3,14,14,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,Office Visit,2370049295,Progress Notes,1,2.338373e+09,1.0,----------------------------------------------...,"[(8741, 8749)]","'ractures, obesity, insulin resistance, dyslip...",['dementia'],1,"[(8641, 8849)]","------- ractures, obesity, insulin resistance,..."
4,15,15,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,Telemedicine - audio only,4959788710,Progress Notes,1,4.951007e+09,1.0,----------------------------------------------...,"[(1048, 1052), (4878, 4887), (5104, 5110), (51...",' accept for this week when he ran out and use...,['MOCA'],1,"[(948, 1152), (4778, 4987), (5004, 5331)]",------- accept for this week when he ran out ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79861,270288,270288,100365175,10059358563,Z8714148,3.336903e+09,2020-12-29 00:00:00.0000000,Telemedicine - audio only,5064411575,Progress Notes,1,5.056437e+09,1.0,----------------------------------------------...,"[(9535, 9544)]","'ogy, Allergy, Immunology Please note -- this ...",['cognition'],1,"[(9435, 9644)]","------- ogy, Allergy, Immunology Please note -..."
79862,270291,270291,100365175,10059358563,Z8714148,3.349162e+09,2021-03-29 00:00:00.0000000,Office Visit,5541979043,Progress Notes,1,5.537594e+09,1.0,----------------------------------------------...,"[(10322, 10331)]","'Physical Exam. Jonathan P March, DO 4/11/2021...",['cognition'],1,"[(10222, 10431)]","------- Physical Exam. Jonathan P March, DO 4/..."
79863,270292,270292,100365175,10059358563,Z8714148,3.360340e+09,2021-04-23 00:00:00.0000000,Telemedicine,5693313879,Progress Notes,1,5.690061e+09,1.0,----------------------------------------------...,"[(964, 970)]",'he past 3 years. Interval events He was last ...,['memory'],1,"[(864, 1070)]",------- he past 3 years. Interval events He wa...
79864,270293,270293,100365175,10059358563,Z8714148,3.363845e+09,2021-05-10 00:00:00.0000000,Office Visit,5784497372,Progress Notes,1,5.781962e+09,1.0,----------------------------------------------...,"[(391, 400)]",' and a moderate mixed hearing loss rising to ...,['cognition'],1,"[(291, 500)]",------- and a moderate mixed hearing loss ris...


In [85]:
def get_match(query, choices, limit = 1):
    results = process.extract(query, choices, limit = limit)
    return results

In [91]:
unique_empi = df["EMPI"].unique().tolist()
unique_empi = unique_empi[:1]

[100000272]


In [102]:
total_merged_sequences = []

for i in unique_empi:
    merged_sequences = df[df["EMPI"] == i]["merged_regex_sent"].values
    #print("MERGED: ", merged_sequences, "\n")

    for j in merged_sequences:
        choices = []
        
        for k in merged_sequences:
            if (k != j):
                choices.append(k)
                
        result = get_match(j, choices)
        score = result[0][1]
        txt = result[0][0]
        
        idx = df[df["merged_regex_sent"] == str(txt)].index[0]
        print(idx)

4
4
4
4
10
0
4
4
4
4
4
4


In [105]:
for i in unique_empi:
    merged_sequences = df[df["EMPI"] == i]["merged_regex_sent"].values
    
    

['------- ncer. She passed secondary to this. He does not know his father. He thinks his father passed from a cerebral hemorrage. His niece had breast cancer diagnosed in her 50s. He had one sister who passed from a'
 '------- a. Skin: No rashes or lesions. Neurological: Awake, alert and oriented x 3. Attention, language and memory are intact. Cranial nerve II-XII are grossly intact. Strength and sensation are grossly intact.'
 '------- cular: Normal rate. Pulmonary/Chest: Effort normal. No stridor. Genitourinary: Penis normal. Uncircumcised. No penile erythema. No discharge found. Musculoskeletal: Normal range of motion. He exhibits'
 '------- ractures, obesity, insulin resistance, dyslipidemia, greater risk for diabetes, CVD and concern for dementia. While the safety of IAD is well established for men starting ADT in the absence of visible'
 '-------  accept for this week when he ran out and used some tramadol. He saw geriatric in November were his MOCA was 14. REVIEW OF SYSTEMS: As 

In [23]:
len(df)

79866

In [28]:
df["PatientEncounterID"].nunique()

50766

In [26]:
df["PatientEncounterID"].isna().sum()

0

In [32]:
# df[df.duplicated(subset=['PatientEncounterID']) == True]

In [37]:
df

,Unnamed: 0,Unnamed: 0.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,NoteID,InpatientNoteTypeDSC,...,NoteCSNID,LineNBR,NoteTXT,regex_location,regex_sent,regex_match,match_count,merged_row_location,merged_regex_sent,unique_encounters
0,0,0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,471433064,Progress Notes,...,4.532520e+08,1.0,----------------------------------------------...,"[(5850, 5858)]",'ncer. She passed secondary to this. He does n...,['cerebral'],1,"[(5750, 5958)]",------- ncer. She passed secondary to this. He...,NaN
1,1,1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,513142600,Progress Notes,...,4.924833e+08,1.0,----------------------------------------------...,"[(3643, 3649)]",'a. Skin: No rashes or lesions. Neurological: ...,['memory'],1,"[(3543, 3749)]",------- a. Skin: No rashes or lesions. Neurolo...,NaN
2,13,13,100000272,10040002098,Z6352398,3.161459e+09,2017-06-23 00:00:00.0000000,Hospital Encounter,1562613299,ED Provider Notes,...,1.508556e+09,1.0,----------------------------------------------...,"[(3947, 3950)]",'cular: Normal rate. Pulmonary/Chest: Effort n...,['mci'],1,"[(3847, 4050)]",------- cular: Normal rate. Pulmonary/Chest: E...,NaN
3,14,14,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,Office Visit,2370049295,Progress Notes,...,2.338373e+09,1.0,----------------------------------------------...,"[(8741, 8749)]","'ractures, obesity, insulin resistance, dyslip...",['dementia'],1,"[(8641, 8849)]","------- ractures, obesity, insulin resistance,...",NaN
4,15,15,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,Telemedicine - audio only,4959788710,Progress Notes,...,4.951007e+09,1.0,----------------------------------------------...,"[(1048, 1052), (4878, 4887), (5104, 5110), (51...",' accept for this week when he ran out and use...,['MOCA'],1,"[(948, 1152), (4778, 4987), (5004, 5331)]",------- accept for this week when he ran out ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79861,270288,270288,100365175,10059358563,Z8714148,3.336903e+09,2020-12-29 00:00:00.0000000,Telemedicine - audio only,5064411575,Progress Notes,...,5.056437e+09,1.0,----------------------------------------------...,"[(9535, 9544)]","'ogy, Allergy, Immunology Please note -- this ...",['cognition'],1,"[(9435, 9644)]","------- ogy, Allergy, Immunology Please note -...",NaN
79862,270291,270291,100365175,10059358563,Z8714148,3.349162e+09,2021-03-29 00:00:00.0000000,Office Visit,5541979043,Progress Notes,...,5.537594e+09,1.0,----------------------------------------------...,"[(10322, 10331)]","'Physical Exam. Jonathan P March, DO 4/11/2021...",['cognition'],1,"[(10222, 10431)]","------- Physical Exam. Jonathan P March, DO 4/...",NaN
79863,270292,270292,100365175,10059358563,Z8714148,3.360340e+09,2021-04-23 00:00:00.0000000,Telemedicine,5693313879,Progress Notes,...,5.690061e+09,1.0,----------------------------------------------...,"[(964, 970)]",'he past 3 years. Interval events He was last ...,['memory'],1,"[(864, 1070)]",------- he past 3 years. Interval events He wa...,NaN
79864,270293,270293,100365175,10059358563,Z8714148,3.363845e+09,2021-05-10 00:00:00.0000000,Office Visit,5784497372,Progress Notes,...,5.781962e+09,1.0,----------------------------------------------...,"[(391, 400)]",' and a moderate mixed hearing loss rising to ...,['cognition'],1,"[(291, 500)]",------- and a moderate mixed hearing loss ris...,NaN


## Data Visualization

In [ ]:
x = ((7509, 7518), (7735, 7741), (7745, 7754), (7858, 7862))
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(str(y))
z = len(p1[(p1["regex_location"] == str(y)) & (p1["PatientEncounterID"] == int(encounters[21]))]['NoteTXT'].values[1])
z

In [ ]:
x = [(181, 390),
  (479, 688),
  (2048, 2269),
  (6184, 6390),
  (6584, 6821),
  (7450, 7659),
  (7716, 7932),
  (9864, 10072),
  (11192, 11648),
  (12118, 12627)]
y = []
for i in range(len(x)):
    a = x[i][0]
    b = x[i][1]
    y.append((a,b))
print(y)

In [ ]:
len(p1[(p1["regex_location"] == "[" +str(subset_loc[1]) + "]") & (p1["PatientEncounterID"] == int(encounters[1]))]['NoteTXT'].values[0])